# AI Research Assistant Agent
This notebook tests the tools we'll use for our AI Research Assistant Capstone Project.

**Tools:**
- SerpAPI: For real-time web search
- Hugging Face Transformers: For text summarization
- Google Colab (notebook environment)

In [1]:
!pip install google-search-results transformers


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=7b2c150ea22abd37120c0da550d450024ece3de3633f0baaeace798118a7a4ec
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


## 🔍 Web Search with SerpAPI
This section performs a Google search using SerpAPI to fetch relevant articles.
We’ll test if SerpAPI is working by searching for articles on facial recognition in law enforcement.

In [3]:
from serpapi import GoogleSearch

params = {
    "q": "Benefits of facial recognition "in policing",
    "api_key": "YOUR_API_KEY"
    "num": 3
}

search = GoogleSearch(params)
results = search.get_dict()

for i, result in enumerate(results.get("organic_results", [])):
    print(f"{i+1}. {result['title']}")
    print(result['link'])
    print()


1. Facial Recognition & Law Enforcement – The Value Proposition
https://www.aware.com/blog-facial-recognition-used-in-law-enforcement/

2. Facial Recognition Technology: Federal Law Enforcement ...
https://www.gao.gov/products/gao-24-107372

3. Understanding The Debate On Facial Recognition ...
https://ace-usa.org/blog/research/understanding-the-debate-on-facial-recognition-technology-in-policing-pros-cons-and-privacy-concerns/



## 🧠 Text Summarization with Hugging Face Transformers
We’ll use the BART model to summarize a sample paragraph related to law enforcement.


In [4]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

text = """
Facial recognition technology is increasingly being adopted by law enforcement agencies around the world.
It offers the ability to quickly identify suspects in public spaces, enhance security, and reduce crime.
However, the use of this technology also raises ethical and privacy concerns, including the risk of wrongful
identification and potential misuse.
"""

summary = summarizer(text, max_length=60, min_length=20, do_sample=False)
print("Summary:", summary[0]['summary_text'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Summary: Facial recognition technology is increasingly being adopted by law enforcement agencies around the world. It offers the ability to quickly identify suspects in public spaces, enhance security, and reduce crime. However, the use of this technology also raises ethical and privacy concerns.


## 🧠 Full Research Assistant Agent Pipeline
This function takes a question, performs a web search, summarizes the results, and returns formatted output.


In [6]:
# Agent memory
agent_memory = {}

def research_agent(query):
    if not query:
        return "Please enter a valid question."

    params = {
        "q": query,
        "api_key": "YOUR_API_KEY",  # Replace with your real key
        "num": 3
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    summaries = []
    sources = []

    for result in results.get("organic_results", []):
        title = result.get("title")
        link = result.get("link")
        snippet = result.get("snippet", "")

        if snippet:
            try:
                summary = summarizer(snippet, max_length=60, min_length=20, do_sample=False)
                summaries.append(summary[0]['summary_text'])
                sources.append(link)
            except Exception as e:
                summaries.append("[Summary failed]")
                sources.append(link)

    # Save to memory
    agent_memory[query] = {
        "summaries": summaries,
        "sources": sources
    }

    output = f"🔍 Research Results for: **{query}**\n\n"
    for i, (summary, link) in enumerate(zip(summaries, sources)):
        output += f"{i+1}. {summary}\n   [Source]({link})\n\n"

    return output if summaries else "No useful results found."


## 🧪 Test the Full Agent
Let's run the research agent with a real example by asking the agent a real research question and see how it responds.


In [ ]:
from IPython.display import Markdown
Markdown(research_agent("How is AI used in criminal investigations?"))

## 🧠 Memory Lookup (Optional)
Check if a question was already asked and retrieve the saved answers.


In [8]:
def check_memory(query):
    if query in agent_memory:
        print("✅ Found in memory:\n")
        for i, (summary, link) in enumerate(zip(agent_memory[query]["summaries"], agent_memory[query]["sources"])):
            print(f"{i+1}. {summary}\n   Source: {link}\n")
    else:
        print("❌ No memory of this question. Run research_agent(query) first.")


In [9]:
!pip freeze > requirements.txt
